In [2]:
from imbed_data_prep.epstein_files import acquire_epstein_data
acquire_epstein_data(
    sqlite_db_file = '/Users/thorwhalen/Dropbox/py/misc/Epstein-doc-explorer/document_analysis.db'
)

Connecting DuckDB and loading sqlite_scanner...
Attaching SQLite database...
Tables discovered: ['canonical_entities', 'documents', 'entity_aliases', 'rdf_triples', 'sqlite_sequence', 'tag_embeddings']
Exporting canonical_entities -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/canonical_entities.parquet
Exporting documents -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/documents.parquet
Exporting entity_aliases -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/entity_aliases.parquet
Exporting rdf_triples -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/rdf_triples.parquet
Exporting sqlite_sequence -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/sqlite_sequence.parquet
Exporting tag_embeddings -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/tag_embeddings.parquet
Exporting rdf_triples edge list -> /Users/thorwhalen/Downloads/epstein_doc_explorer_parquet/relationships_edges.parquet
Done. Output folder: /Users/thorwhal

PosixPath('/Users/thorwhalen/Downloads/epstein_doc_explorer_parquet')